In [33]:
# dataset generation
import pandas as pd
import numpy as np
import category_encoders as ce

# Simulating a dataset
data = {
    'Age': np.random.randint(20, 60, size=100).astype(float),  # Random ages between 20 and 60
    'State': np.random.choice(['Karnataka', 'Tamil Nadu', 'Maharashtra', 'Delhi', 'Telangana'], size=100),
    'Education': np.random.choice(['High School', 'UG', 'PG'], size=100),
    'Package': np.random.rand(100) * 100  # Random package values for demonstration
}

# Introducing missing values in 'Age' column (5%)
np.random.seed(0)  # For reproducibility
missing_indices = np.random.choice(data['Age'].shape[0], replace=False, size=int(data['Age'].shape[0] * 0.05))
data['Age'][missing_indices] = np.nan

df = pd.DataFrame(data)

df.head()

,Age,State,Education,Package
0,36.0,Karnataka,PG,27.165277
1,43.0,Delhi,PG,45.544415
2,NaN,Tamil Nadu,UG,40.171354
3,24.0,Telangana,UG,24.841347
4,56.0,Delhi,UG,50.586638


In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Package']), df['Package'], test_size=0.2, random_state=42)

In [35]:
X_train.head()

,Age,State,Education
55,NaN,Karnataka,UG
88,48.0,Karnataka,High School
26,NaN,Telangana,PG
42,54.0,Tamil Nadu,High School
69,27.0,Delhi,High School


In [36]:
X_train['State'].value_counts()

,count
State,
Maharashtra,21
Tamil Nadu,20
Telangana,15
Delhi,14
Karnataka,10


In [37]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import sklearn

In [38]:
class CountEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns
        self.count_map = {}

    def fit(self, X, y=None):
        if self.columns is None:
            self.columns = X.columns
        for col in self.columns:
            self.count_map[col] = X[col].value_counts().to_dict()
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            X[col] = X[col].map(self.count_map[col]).fillna(0)
        return X

In [39]:
preprocessor = ColumnTransformer(
    transformers=[
        ('age_missing', SimpleImputer(strategy='mean'), ['Age']),
        ('cat_state', CountEncoder(), ['State']),
        ('education_ordinal', OrdinalEncoder(), ['Education'])
    ])

sklearn.set_config(transform_output="pandas")

In [40]:
preprocessor.fit_transform(X_train)

,age_missing__Age,cat_state__State,education_ordinal__Education
55,41.013333,10,2.0
88,48.000000,10,0.0
26,41.013333,15,1.0
42,54.000000,20,0.0
69,27.000000,14,0.0
...,...,...,...
60,46.000000,10,1.0
71,41.000000,14,1.0
14,53.000000,15,2.0
92,45.000000,20,0.0


In [41]:
# using category encoders
from category_encoders.count import CountEncoder

In [42]:
preprocessor = ColumnTransformer(
    transformers=[
        ('age_missing', SimpleImputer(strategy='mean'), ['Age']),
        ('cat_state', CountEncoder(normalize=True), ['State']), # parameter - normalize = True is a new transformer called FrequencyEncoder | CountEncoder
        ('education_ordinal', OrdinalEncoder(), ['Education'])
    ])
sklearn.set_config(transform_output="pandas")

In [43]:
preprocessor.fit_transform(X_train)

,age_missing__Age,cat_state__State,education_ordinal__Education
55,41.013333,0.1250,2.0
88,48.000000,0.1250,0.0
26,41.013333,0.1875,1.0
42,54.000000,0.2500,0.0
69,27.000000,0.1750,0.0
...,...,...,...
60,46.000000,0.1250,1.0
71,41.000000,0.1750,1.0
14,53.000000,0.1875,2.0
92,45.000000,0.2500,0.0


In [45]:
# parameters
import pandas as pd
import numpy as np
import category_encoders as ce

# Simulating a dataset
np.random.seed(42)  # For reproducibility
data = {
    'State': np.random.choice(['Karnataka', 'Tamil Nadu', 'Maharashtra', 'Delhi', 'Telangana', None], size=100),
    'Education': np.random.choice(['High School', 'UG', 'PG', None], size=100)
}
df = pd.DataFrame(data)

df.head(25)


,State,Education
0,Delhi,PG
1,Telangana,High School
2,Maharashtra,High School
3,Telangana,High School
4,Telangana,PG
5,Tamil Nadu,High School
6,Maharashtra,None
7,Maharashtra,High School
8,Maharashtra,None
9,Telangana,None


In [46]:
df.isnull().sum()

,0
State,17
Education,23


In [47]:
# Initialize the CountEncoder with various parameters
encoder = ce.CountEncoder(
    cols=['State', 'Education'],  # Specify columns to encode. None would automatically select all categorical columns.
    handle_missing='return_nan',  # Treat NaNs as a countable category. Options are ‘error’, ‘return_nan’, and ‘value’. Default ‘value’,
    handle_unknown='return_nan',  # Treat unknown categories as NaNs (if seen during transform but not in fit). Options are ‘error’ ‘return_nan’, ‘value’ and int.
)

In [48]:
# Fit and transform the dataset
encoder.fit_transform(df)

,State,Education
0,25.0,34.0
1,17.0,27.0
2,11.0,27.0
3,17.0,27.0
4,17.0,34.0
...,...,...
95,25.0,27.0
96,25.0,16.0
97,17.0,NaN
98,11.0,NaN


In [49]:
encoder.mapping

{'State': State
 Delhi          25.0
 Tamil Nadu     19.0
 Telangana      17.0
 NaN             NaN
 Maharashtra    11.0
 Karnataka      11.0
 Name: count, dtype: float64,
 'Education': Education
 PG             34.0
 High School    27.0
 NaN             NaN
 UG             16.0
 Name: count, dtype: float64}

In [50]:
new_data = pd.DataFrame({'State': ['Bihar'], 'Education': ['UG']}) # New State
encoder.transform(new_data)

,State,Education
0,NaN,16.0


In [51]:
np.random.seed(0)  # For reproducibility
data = {
    'Category': np.random.choice(['A', 'B', 'C', 'D', 'E', 'F', np.nan], size=100, p=[0.3, 0.25, 0.15, 0.15, 0.05, 0.05, 0.05]),
    'Value': np.random.rand(100)
}

df = pd.DataFrame(data)

df.sample(10)


,Category,Value
91,C,0.209844
29,B,0.290078
2,C,0.735194
50,C,0.149448
44,C,0.806194
78,A,0.704414
33,C,0.298282
65,B,0.855803
75,A,0.223925
45,C,0.703889


In [52]:
df['Category'].value_counts()

,count
Category,
A,34
B,22
C,21
D,12
nan,5
F,4
E,2


In [53]:
encoder = ce.CountEncoder(
    cols=['Category'],
    min_group_size=10,  # Groups with counts less than 5 will be combined
    # min_group_name='salman',  # Use default naming for combined minimum groups
)

# Fit and transform the dataset
encoded_df = encoder.fit_transform(df['Category'])

# Display the original and encoded data for comparison
df['Encoded'] = encoded_df
print(df.head(20))

   Category     Value  Encoded
0         B  0.677817       22
1         D  0.270008       12
2         C  0.735194       21
3         B  0.962189       22
4         B  0.248753       22
5         C  0.576157       21
6         B  0.592042       22
7         E  0.572252       11
8       nan  0.223082       11
9         B  0.952749       22
10        D  0.447125       12
11        B  0.846409       22
12        C  0.699479       21
13        F  0.297437       11
14        A  0.813798       34
15        A  0.396506       34
16        A  0.881103       34
17        D  0.581273       12
18        D  0.881735       12
19        E  0.692532       11


In [54]:
encoder.mapping

{'Category': Category
 A          34
 B          22
 C          21
 D          12
 E_F_nan    11
 Name: count, dtype: int64}

### Binary Encoder

In [55]:
import pandas as pd
import category_encoders as ce

# Sample dataset
data = {
    'Item': ['Item1', 'Item2', 'Item3', 'Item4', 'Item5', 'Item6', 'Item7', 'Item8'],
    'Fruit': ['Apple', 'Banana', 'Cherry', 'Date', 'Elderberry', 'Fig', 'Grape', 'Honeydew']
}
df = pd.DataFrame(data)
df

,Item,Fruit
0,Item1,Apple
1,Item2,Banana
2,Item3,Cherry
3,Item4,Date
4,Item5,Elderberry
5,Item6,Fig
6,Item7,Grape
7,Item8,Honeydew


In [56]:
# Initialize the Binary Encoder
encoder = ce.BinaryEncoder(cols=['Fruit'], return_df=True)

# Fit and transform the data
df_encoded = encoder.fit_transform(df)

# Display the original and encoded data
print(df_encoded)

    Item  Fruit_0  Fruit_1  Fruit_2  Fruit_3
0  Item1        0        0        0        1
1  Item2        0        0        1        0
2  Item3        0        0        1        1
3  Item4        0        1        0        0
4  Item5        0        1        0        1
5  Item6        0        1        1        0
6  Item7        0        1        1        1
7  Item8        1        0        0        0


The problemof using this encoder technique

You cannot use this technique with `interpretation problme` because the columns which was created after transformation has no interperation ability.

### Target Encoder

In [57]:
# using category_encoder

import pandas as pd
import category_encoders as ce

# Sample data
data = {
    'Feature': ['A', 'B', 'A', 'B', 'C', 'A', 'B', 'C'],
    'Target': [1, 0, 0, 1, 1, 1, 0, 1]
}
df = pd.DataFrame(data)

# Separating the feature and target columns
X = df.drop('Target', axis=1)
y = df['Target']

# Initialize the TargetEncoder
encoder = ce.TargetEncoder(cols=['Feature'])

# Fit the encoder using the feature data and target variable
encoder.fit(X, y)

# Transform the data
encoded = encoder.transform(X)

# Show the original and encoded data
print(pd.concat([df, encoded], axis=1))


   Feature  Target   Feature
0        A       1  0.631436
1        B       0  0.579948
2        A       0  0.631436
3        B       1  0.579948
4        C       1  0.678194
5        A       1  0.631436
6        B       0  0.579948
7        C       1  0.678194


In [58]:
encoder.mapping

{'Feature': Feature
  1    0.631436
  2    0.579948
  3    0.678194
 -1    0.625000
 -2    0.625000
 dtype: float64}

In [59]:
# using sklearn
import pandas as pd
from sklearn.preprocessing import TargetEncoder

# Sample data
data = {
    'Feature': ['A', 'B', 'A', 'B', 'C', 'A', 'B', 'C'],
    'Target': [1, 0, 0, 1, 1, 1, 0, 1]
}
df = pd.DataFrame(data)

# Separating the feature and target columns
X = df.drop('Target', axis=1)
y = df['Target']

# Initialize the TargetEncoder
encoder = TargetEncoder(smooth=0.0)

# Fit the encoder using the feature data and target variable
encoder.fit(X, y)

# Transform the data
encoded = encoder.transform(X)
encoded

,Feature
0,0.666667
1,0.333333
2,0.666667
3,0.333333
4,1.000000
5,0.666667
6,0.333333
7,1.000000


### Weight of Evidence

In [60]:
import pandas as pd
import category_encoders as ce

# Example dataset
data = {
    'Feature': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'A', 'C'],
    'Target': [1, 0, 0, 1, 1, 0, 1, 0, 1, 0]
}
df = pd.DataFrame(data)

# Define the features and target
X = df[['Feature']]
y = df['Target']

# Initialize and fit the TargetEncoder
encoder = ce.WOEEncoder(cols=['Feature'])
X_encoded = encoder.fit_transform(X, y)

# Display the original and encoded data
df['Feature_Encoded'] = X_encoded
print(df)


  Feature  Target  Feature_Encoded
0       A       1         0.000000
1       B       0        -0.405465
2       A       0         0.000000
3       C       1         0.405465
4       B       1        -0.405465
5       A       0         0.000000
6       C       1         0.405465
7       B       0        -0.405465
8       A       1         0.000000
9       C       0         0.405465
